# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
# import libraries
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger','stopwords'])
import pandas as pd 
import numpy as np
import re
import pickle
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.metrics import f1_score, classification_report, accuracy_score, make_scorer
from scipy.stats.mstats import gmean
from sklearn.model_selection import GridSearchCV
from sklearn.base import BaseEstimator, TransformerMixin
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/tuannguyen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/tuannguyen/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/tuannguyen/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tuannguyen/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///../data/DisasterResponse.db')
df = pd.read_sql_table('messages',engine)
X = df['message']
Y = df.drop(['message','genre','original','id'],axis=1)

In [3]:
Y.columns

Index(['related', 'request', 'offer', 'aid_related', 'medical_help',
       'medical_products', 'search_and_rescue', 'security', 'military',
       'child_alone', 'water', 'food', 'shelter', 'clothing', 'money',
       'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    text = re.sub('[^a-zA-Z0-9]',' ',text)
    words = word_tokenize(text)
    words = [w for w in words if w not in stopwords.words('english')]
    
    lemmatizer = WordNetLemmatizer()
    clean_words = []
    for word in words: 
        clean_word = lemmatizer.lemmatize(word).lower().strip()
        clean_words.append(clean_word)
    return clean_words

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [19]:
pipeline = Pipeline([
    ('vect',CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('clf',MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X,Y)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                  

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [7]:
y_pred = pipeline.predict(X_test)

In [8]:
category_names = Y.columns
for i in range(len(category_names)):
    print('-'*60,'\n',"Category:", category_names[i],"\n", classification_report(y_test.iloc[:, i].values, y_pred[:, i]))
    print('Accuracy of',category_names[i], accuracy_score(y_test.iloc[:, i].values, y_pred[:,i]))


------------------------------------------------------------ 
 Category: related 
               precision    recall  f1-score   support

           0       0.64      0.47      0.54      1569
           1       0.84      0.92      0.88      4941
           2       0.79      0.25      0.38        44

    accuracy                           0.81      6554
   macro avg       0.76      0.54      0.60      6554
weighted avg       0.79      0.81      0.79      6554

Accuracy of related 0.8051571559353067
------------------------------------------------------------ 
 Category: request 
               precision    recall  f1-score   support

           0       0.89      0.98      0.93      5439
           1       0.78      0.41      0.54      1115

    accuracy                           0.88      6554
   macro avg       0.84      0.69      0.74      6554
weighted avg       0.87      0.88      0.87      6554

Accuracy of request 0.8808361306072627
------------------------------------------------

Accuracy of electricity 0.9806225205981081
------------------------------------------------------------ 
 Category: tools 
               precision    recall  f1-score   support

           0       0.99      1.00      1.00      6512
           1       0.00      0.00      0.00        42

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      0.99      0.99      6554

Accuracy of tools 0.9935916997253585
------------------------------------------------------------ 
 Category: hospitals 
               precision    recall  f1-score   support

           0       0.99      1.00      1.00      6500
           1       0.00      0.00      0.00        54

    accuracy                           0.99      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.98      0.99      0.99      6554

Accuracy of hospitals 0.9917607567897467
-----------------------------------------------------------

In [9]:
def multioutput_f1score(y_true, y_pred): 
    scores = []
    for i in range(Y.shape[1]): 
        score = f1_score(y_true.iloc[:,i], y_pred[:,i],average='weighted')
        scores.append(score)
    scores = np.asarray(scores)
    score = gmean(scores)
    return score

In [10]:
multioutput_f1score(y_test,y_pred)

0.929668191609448

### 6. Improve your model
Use grid search to find better parameters. 

In [20]:
parameters = {
    'clf__estimator__n_estimators':[100,200],
    'clf__estimator__min_samples_split':[2,3,4],
    'clf__estimator__criterion': ['entropy', 'gini']
}

scorer = make_scorer(multioutput_f1score,greater_is_better = True)

cv = GridSearchCV(pipeline,param_grid=parameters,verbose = 2, n_jobs = -1)
cv.fit(X_train, y_train)

Fitting 3 folds for each of 12 candidates, totalling 36 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  32 out of  36 | elapsed: 49.2min remaining:  6.2min
[Parallel(n_jobs=-1)]: Done  36 out of  36 | elapsed: 50.0min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        CountVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.int64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                       

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [21]:
cv.best_estimator_

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at...
                 MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True,
                                                                        class_weight=None,
                                                                        criterion='gini',
                                                                  

In [22]:
y_pred = cv.best_estimator_.predict(X_test)

multioutput_f1score(y_test,y_pred)

0.9354670168728948

In [23]:
for i in range(len(category_names)):
    print('-'*60,'\n',"Category:", category_names[i],"\n", classification_report(y_test.iloc[:, i].values, y_pred[:, i]))
    print('Accuracy of',category_names[i], accuracy_score(y_test.iloc[:, i].values, y_pred[:,i]))

------------------------------------------------------------ 
 Category: related 
               precision    recall  f1-score   support

           0       0.73      0.44      0.55      1569
           1       0.84      0.95      0.89      4941
           2       0.70      0.32      0.44        44

    accuracy                           0.82      6554
   macro avg       0.75      0.57      0.62      6554
weighted avg       0.81      0.82      0.80      6554

Accuracy of related 0.820872749465975
------------------------------------------------------------ 
 Category: request 
               precision    recall  f1-score   support

           0       0.90      0.98      0.94      5439
           1       0.82      0.49      0.61      1115

    accuracy                           0.90      6554
   macro avg       0.86      0.73      0.78      6554
weighted avg       0.89      0.90      0.88      6554

Accuracy of request 0.8950259383582545
-------------------------------------------------

------------------------------------------------------------ 
 Category: infrastructure_related 
               precision    recall  f1-score   support

           0       0.94      1.00      0.97      6142
           1       0.00      0.00      0.00       412

    accuracy                           0.94      6554
   macro avg       0.47      0.50      0.48      6554
weighted avg       0.88      0.94      0.91      6554

Accuracy of infrastructure_related 0.9365273115654562
------------------------------------------------------------ 
 Category: transport 
               precision    recall  f1-score   support

           0       0.95      1.00      0.97      6217
           1       0.79      0.06      0.11       337

    accuracy                           0.95      6554
   macro avg       0.87      0.53      0.54      6554
weighted avg       0.94      0.95      0.93      6554

Accuracy of transport 0.950717119316448
------------------------------------------------------------ 
 Catego

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [12]:
pipeline = Pipeline([
    ('vect',CountVectorizer(tokenizer=tokenize)),
    ('tfidf',TfidfTransformer()),
    ('clf',MultiOutputClassifier(AdaBoostClassifier()))
])

In [13]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at 0x1a1bc15f28>,
                                 vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME.R',


In [14]:
y_pred = pipeline.predict(X_test)

In [15]:
multioutput_f1score(y_test,y_pred)

0.9363827443400324

In [16]:
for i in range(len(category_names)):
    print('-'*60,'\n',"Category:", category_names[i],"\n", classification_report(y_test.iloc[:, i].values, y_pred[:, i]))
    print('Accuracy of',category_names[i], accuracy_score(y_test.iloc[:, i].values, y_pred[:,i]))

------------------------------------------------------------ 
 Category: related 
               precision    recall  f1-score   support

           0       0.67      0.11      0.19      1569
           1       0.77      0.98      0.87      4941
           2       0.68      0.30      0.41        44

    accuracy                           0.77      6554
   macro avg       0.71      0.46      0.49      6554
weighted avg       0.75      0.77      0.70      6554

Accuracy of related 0.7689960329569728
------------------------------------------------------------ 
 Category: request 
               precision    recall  f1-score   support

           0       0.90      0.97      0.93      5439
           1       0.74      0.50      0.60      1115

    accuracy                           0.89      6554
   macro avg       0.82      0.73      0.76      6554
weighted avg       0.88      0.89      0.88      6554

Accuracy of request 0.8854134879462924
------------------------------------------------

Accuracy of hospitals 0.9900823924321025
------------------------------------------------------------ 
 Category: shops 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00      6531
           1       0.00      0.00      0.00        23

    accuracy                           1.00      6554
   macro avg       0.50      0.50      0.50      6554
weighted avg       0.99      1.00      0.99      6554

Accuracy of shops 0.9954226426609704
------------------------------------------------------------ 
 Category: aid_centers 
               precision    recall  f1-score   support

           0       0.99      1.00      0.99      6485
           1       0.22      0.07      0.11        69

    accuracy                           0.99      6554
   macro avg       0.60      0.53      0.55      6554
weighted avg       0.98      0.99      0.98      6554

Accuracy of aid_centers 0.9874885566066525
---------------------------------------------------------

### 9. Export your model as a pickle file

In [17]:
pickle.dump(pipeline, open('../models/nb_classifier.pkl', "wb"))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.